In [1]:
import json
import tensorflow as tf
from keras.preprocessing.image import ImageDataGenerator
from keras.preprocessing import image
import numpy as np
import matplotlib.pyplot as plt
from results import Results

In [ ]:
def save(model, filename: str, model_name: str):
    """Saves the model to an .h5 file and the model name to a .json file.

    Args:
        model: Model to be saved
        filename: Relative path to the file without the extension.
        
    """
    # Save Keras model
    model.save(filename + '.h5')

    # Save base model information
    with open(filename + '.json', 'w', encoding='utf-8') as f:
        json.dump(model_name, f, ensure_ascii=False, indent=4, sort_keys=True)
def load(self, filename: str):
    """Loads a trained CNN model and the corresponding preprocessing information.

    Args:
        filename: Relative path to the file without the extension.

    """
    # Load Keras model
    self._model = tf.keras.models.load_model(filename + '.h5')

    # Load base model information
    with open(filename + '.json') as f:
        self._model_name = json.load(f)

    self._initialize_attributes()

In [ ]:
train_datagen = ImageDataGenerator(
        # reducing/normalizing the pixels
        rescale=1./255,
        shear_range=0.2,
        zoom_range=0.2) 
#We dont want the dataset to be flipped as orientation is important, so we delete the horizontal_flip parameter
#connecting the image augmentation tool to our dataset

targetSizeWidth = 112
targetSizeHeight = 112
targetSize = (targetSizeWidth, targetSizeHeight)

train_set = train_datagen.flow_from_directory(
        './training',
        target_size = targetSize,
        batch_size=128,
        class_mode='categorical')

#only rescaling but no transformations
validation_datagen = ImageDataGenerator(rescale=1./255)
#connecting to the test data
val_set = validation_datagen.flow_from_directory(
        './validation',
        target_size = targetSize,
        batch_size = 128,
        class_mode = 'categorical')
#only rescaling but no transformations
test_datagen = ImageDataGenerator(rescale=1./255)
#connecting to the test data
test_set = test_datagen.flow_from_directory(
        './test',
        target_size = targetSize,
        batch_size = 128,
        class_mode = 'categorical')

print(test_set)

In [ ]:
def _plot_training(history):
    """Plots the evolution of the accuracy and the loss of both the training and validation sets.

    Args:
        history: Training history.

    """
    training_accuracy = history.history['accuracy']
    validation_accuracy = history.history['val_accuracy']
    loss = history.history['loss']
    val_loss = history.history['val_loss']
    epochs = range(len(training_accuracy))

    # Accuracy
    plt.figure()
    plt.plot(epochs, training_accuracy, 'r', label='Training accuracy')
    plt.plot(epochs, validation_accuracy, 'b', label='Validation accuracy')
    plt.title('Training and validation accuracy')
    plt.xlabel('Epoch')
    plt.ylabel('Accuracy')
    plt.legend()

    # Loss
    plt.figure()
    plt.plot(epochs, loss, 'r', label='Training loss')
    plt.plot(epochs, val_loss, 'b', label='Validation loss')
    plt.title('Training and validation loss')
    plt.xlabel('Epoch')
    plt.ylabel('Loss')
    plt.legend()

    plt.show()


In [ ]:
#--------------------- Building CNN --------------------#
# initializing CNN as sequential layers
from tf.keras.callbacks import EarlyStopping
def create_model():
        
    cnn = tf.keras.models.Sequential()
    kernel_initializer = tf.keras.initializers.RandomNormal(stddev=0.02)
    # Step 1: Convolution to get the Feature Map
    cnn.add(tf.keras.layers.Conv2D(filters= 64 , kernel_size = 2, activation = 'relu', input_shape=[112,112,3], kernel_initializer=kernel_initializer))
    # filters: output feature map
    # kernel_size: size of the feature detector
    # strides: step size from one filter to the next default is 1
    # Step 2: Max Pooling
    cnn.add(tf.keras.layers.MaxPool2D(pool_size=2 ,strides=2))
    #adding a second convolutional layer
    cnn.add(tf.keras.layers.Conv2D(filters = 64, kernel_size = 2,padding='same', activation = 'relu'), kernel_initializer=kernel_initializer)
    cnn.add(tf.keras.layers.MaxPool2D(pool_size=2 ,strides=2))
    #adding a second convolutional layer
    cnn.add(tf.keras.layers.Conv2D(filters = 128, kernel_size = 2,padding='same', activation = 'relu'), kernel_initializer=kernel_initializer)
    cnn.add(tf.keras.layers.MaxPool2D(pool_size=2 ,strides=2))
    #adding a second convolutional layer
    cnn.add(tf.keras.layers.Conv2D(filters = 128, kernel_size = 2,padding='same', activation = 'relu'), kernel_initializer=kernel_initializer)
    cnn.add(tf.keras.layers.MaxPool2D(pool_size=2 ,strides=2))
    # Step 3: Flattening
    cnn.add(tf.keras.layers.Flatten())

    # Step 4: Full Connection
    cnn.add(tf.keras.layers.Dense(units = 256, activation = 'relu'))

    # Step 5: Output Layer
    cnn.add(tf.keras.layers.Dense(units = 1, activation = 'sigmoid'))
    return cnn

#--------------------- Training the CNN --------------------#
#compiling the CNN
cnn=create_model()
cnn.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])
#training the CNN on the training set and evaluating it on the test set
# cnn.fit(x = train_set, validation_data = test_set, epochs = 25)
epochs=10
# callbacks=
# Train the network
print("\n\nTraining CNN...")
# Monitor in real-time the training and validation accuracy and loss
early_stopping_monitor = EarlyStopping(patience=4)
history = cnn.fit(
    train_set,
    epochs=epochs,
    steps_per_epoch=len(train_set),
    validation_data=val_set,
    validation_steps=len(val_set),
    callbacks=[early_stopping_monitor]
    #callbacks=callbacks
)
if epochs > 1:
    _plot_training(history)


In [ ]:
save(cnn,'./TRModel','CNN_trainingModel')

In [ ]:
def predict(model,test_generator,dataset_name:str, save: bool = True):
    """Evaluates a new set of images using the trained CNN.

    Args:
        test_dir: Relative path to the validation directory (e.g., 'dataset/test').
        dataset_name: Dataset descriptive name.
        save: Save results to an Excel file.

    """
    # Configure loading and pre-processing functions
    print('Reading test data...')
    # test_datagen = tf.keras.preprocessing.image.ImageDataGenerator(preprocessing_function=_preprocessing_function)

    # test_generator = test_datagen.flow_from_directory(
    #     test_dir,
    #     target_size=len(test_dir),
    #     batch_size=1,  # A batch size of 1 ensures that all test images are processed
    #     class_mode='categorical',
    #     shuffle=False
    # )

    # Predict categories
    predictions =model.predict(test_generator)
    predicted_labels = np.argmax(predictions, axis=1).ravel().tolist()

    # Format results and compute classification statistics
    results = Results(test_generator.class_indices, dataset_name=dataset_name)
    accuracy, confusion_matrix, classification = results.compute(test_generator.filenames, test_generator.classes,
                                                                    predicted_labels)
    # Display and save results
    results.print(accuracy, confusion_matrix)

    if save:
        results.save(confusion_matrix, classification, predictions)

In [ ]:
predict(cnn,test_set,'test_1',True)

In [ ]:
from keras.utils import load_img, img_to_array
#--------------------- Single prediction with CNN --------------------#
test_image = load_img('./test/JuanO/testJuanO0.jpg', target_size = (112, 112))
# to convert image in pii format into a numpy array format
test_image = img_to_array(test_image)
# adding extra dimension to put this image into a batch by saying where we want to add this batch (as the first dimension)
test_image = np.expand_dims(test_image, axis = 0)
# cnn prediction on the test image
result = cnn.predict(test_image)
# getting the results encoding: which indices correspond to which classes (1: dog, 0:cat)
print(train_set.class_indices)

#prediction for the single image/element from the batch
if result[0][0] == 0:
   prediction = 'JuanM'
elif result[0][0] == 1:
   prediction = 'JuanO'
else:
   prediction = 'Pablo'

print(prediction)


test_image2 = load_img('./test/Pablo/testPablo0.jpg', target_size = (640, 480))
# to convert image in pii format into a numpy array format
test_image2 = img_to_array(test_image2)
# adding extra dimension to put this image into a batch by saying where we want to add this batch (as the first dimension)
test_image2 = np.expand_dims(test_image2, axis = 0)
# cnn prediction on the test image
result2 = cnn.predict(test_image2)

#prediction for the single image/element from the batch
if result[0][0] == 0:
   prediction = 'JuanM'
elif result[0][0] == 1:
   prediction = 'JuanO'
else:
   prediction = 'Pablo'

print(prediction)